# Script that copies demand excel files from gdrive into bq tables

### imports and auth

In [1]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import re
# from google.colab import auth
# auth.authenticate_user()
# print('Authenticated')

# from google.colab import drive
# drive.mount('/content/drive')

### Declare Variables

In [2]:
params = {
    'gdrive_folder_id' : '1FumjvqmkvENsulhJzOQvNN5KGX7EeVNn', # the ID of gdrive folder to get data from
    'local_download_path' : '/content/data', # the path to download the gdrive data to e.g. '/content/data'
    'sheet_names' : ['HW','TRUG','BUN'], # These are the sheet names for each spreadsheet to read in as pandas dataframe
}

### Get data from gdrive

In [3]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser(params['local_download_path'])
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': f"'{params['gdrive_folder_id']}' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

# with open(fname, 'r') as f:
#   print(f.read())

title: 35 week Primal Advice 10-01-2022 with unlimited capasity.xlsx, id: 10Xupge7xsfs9wDf3n3u44LJ1H5v9vaOY
downloading to /content/data/35 week Primal Advice 10-01-2022 with unlimited capasity.xlsx
title: 35 week Primal Advice 02-12-2021 with unlimited capasity.xlsx, id: 1JegKdGIueeSCOSoy2rqI32NmT6LVhxFC
downloading to /content/data/35 week Primal Advice 02-12-2021 with unlimited capasity.xlsx
title: 35 week Primal Advice 02-09-2021 with unlimited capasity.xlsx, id: 1MD2TS8DqedI9if3GmZeoadZdrvEAh8He
downloading to /content/data/35 week Primal Advice 02-09-2021 with unlimited capasity.xlsx
title: 35 week Primal Advice 28-01-2022 with unlimited capasity.xlsx, id: 1KQu7jNC-bGjd0cIbSCgeTjdk9WYsXpQ5
downloading to /content/data/35 week Primal Advice 28-01-2022 with unlimited capasity.xlsx
title: 35 week Primal Advice 23-09-2021 with unlimited capasity.xlsx, id: 1jtSve24ib-6FY6gmnAEXh8f0Mcg3O9Zk
downloading to /content/data/35 week Primal Advice 23-09-2021 with unlimited capasity.xlsx
title

### from excel to pandas dataframes

In [4]:
df_dict = {}
for f in os.listdir(params['local_download_path']):
  f_path = params['local_download_path']+'/'+f
  df_dict[f] = pd.read_excel(f_path, sheet_name=params['sheet_names'])


### Add columns to each dataframe

In [5]:
#df_dict['35 week Primal Advice 09-09-2021 with unlimited capasity.xlsx']#['HW']#.describe()
# for each sheet of data in excel file (now dict of dataframes in dict where key = filename and subkey = sheetName)
# add a column for sheet name e.g. HW, TRUG, BUN

def get_dmy_from_filename(fn):
  title_search = re.search('([0-9]{2})-([0-9]{2})-([0-9]{4})', fn, re.IGNORECASE)
  if title_search:
    return title_search.group(3)+'-'+title_search.group(2)+'-'+title_search.group(1)
  else:
    return ''

for ss in df_dict.keys():
  for sname in df_dict[ss].keys():
    df_dict[ss][sname]['sheet_name'] = sname
    df_dict[ss][sname]['source_date'] = get_dmy_from_filename(ss)
    df_dict[ss][sname]['file_name'] = ss


### make single dataframe

In [6]:
def stack_df(dict_of_df):
  return pd.concat(dict_of_df).reset_index().drop(columns=['level_0','level_1'])

all_df = pd.concat([stack_df(df_dict[ss]) for ss in df_dict.keys()])

### clean column names

In [7]:
all_df.columns = all_df.columns.str.replace(' ','_', regex=False)
all_df.columns = all_df.columns.str.replace('.','_', regex=False)

In [8]:
all_df.columns

Index(['Date', 'PrimalID', 'Family', 'PrimalName', 'Primal_kgs_for_production',
       'Proposed_Purchases', 'Confirmed_Purchases', 'Low_Codes', 'Op_Stock',
       'Unmature_stock', 'Butcher_shop_Rq_', 'WOW_code', 'Plant', 'sheet_name',
       'source_date', 'file_name', 'Target_inventory', 'TargetInDays',
       'StockInDays'],
      dtype='object')

### reorder columns

In [9]:
# Stefan Petiq 20220214:
# Everything exactly how it is except
# Wow_code
# Plant
# Sheet_name
# Source_date
# file_name
# moved to be the last 5 fields at the end of the sheet

last_cols = ['WOW_code', 'Plant', 'sheet_name', 'source_date', 'file_name']
new_col_order = [x for x in all_df.columns if x not in last_cols]+last_cols

all_df = all_df[new_col_order]

### Delete unwanted sheet

In [10]:
# Delete 'week' from the "35 week Primal Advice 10-01-2022 with unlimited capasity.xlsx" file so it should not appear in the data loading anymore
all_df = all_df.query("file_name != '35 week Primal Advice 10-01-2022 with unlimited capasity.xlsx'")

### change dtypes

In [12]:
all_df.columns

Index(['Date', 'PrimalID', 'Family', 'PrimalName', 'Primal_kgs_for_production',
       'Proposed_Purchases', 'Confirmed_Purchases', 'Low_Codes', 'Op_Stock',
       'Unmature_stock', 'Butcher_shop_Rq_', 'Target_inventory',
       'TargetInDays', 'StockInDays', 'WOW_code', 'Plant', 'sheet_name',
       'source_date', 'file_name'],
      dtype='object')

In [15]:
def float2str(d):
  return ['' if np.isnan(x) else str(int(x)) for x in d]
all_df['WOW_code'] = float2str(all_df['WOW_code'])
all_df['PrimalID'] = float2str(all_df['PrimalID'])

### save to bigquery

In [18]:
pd.io.gbq.to_gbq(all_df, 'masterdata_view.hfa_primal_advice', 'gcp-wow-pvc-grnstck-prod', chunksize=100000, reauth=False, if_exists='replace')

12it [04:47, 24.00s/it]
